# Arm Reacher

---

In this notebook, we're training an a robotic arm to track a moving target (blue-green sphere). 

We use the Unity ML-Agents environment. This is performed as part of a project for the course [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions (README) to double-check that you have installed the necessary packages. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from unityagents import UnityEnvironment

from collaborative_agent import CollaborativeAgent
from train import train

%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/psankar/miniconda3/envs/arm_reacher/lib/python3.6/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


Next, we will start the environment. **Note that this will only work on a Linux (x64) machine.**

In [2]:
env = UnityEnvironment(file_name="../unity_environment/Tennis.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


You should now see a Unity visualization window just spawned. If you get any pop-ups on the application not responding, just click "Wait" or ignore the message. 

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# Reset the environment.
env_info = env.reset(train_mode=False)[brain_name]

# Number of agents.
num_agents = len(env_info.agents)
print(f'Number of agents: {num_agents}')

# Size of action. 
action_size = brain.vector_action_space_size
print(f'Number of possible actions: {action_size}')

# Size of state. 
states = env_info.vector_observations
state_size = states.shape[1]
print(f'Number of possible states: {state_size}')

Number of agents: 2
Number of possible actions: 2
Number of possible states: 24


### 3. Take Random Actions in the Environment

Let's confirm that we have everything set up and the visualization works. 

Here, we will watch the agents' (robot arms) performance, as they select actions (uniformly) at random with each time step.  A window should pop up that allows you to observe the agents, as they move through the environment.  

In [5]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

### 4. Let's train the arm!

Let's construct our robotic arm from the `Agent` class. 

In [6]:
agent = CollaborativeAgent(num_agents=num_agents, observation_size=state_size, action_size=action_size, device=device, seed=0)

Let's begin training. 

In [7]:
NUM_EPISODES = 5000                  # Maximum number of training episodes.
MAX_TIME_IN_EPISODE = 1000            # Maximum number of timesteps per episode.
SCORE_ACCEPTANCE_THRESHOLD = 0.5     # The average score of all agents for the task to be considered successful. 

In [ ]:
scores = train(env, 
               brain_name, 
               agent, 
               NUM_EPISODES, 
               MAX_TIME_IN_EPISODE,
               SCORE_ACCEPTANCE_THRESHOLD)

Episode 100	Average Score: 0.000000
Episode 200	Average Score: 0.000000
Episode 300	Average Score: 0.007000
Episode 400	Average Score: 0.001000
Episode 500	Average Score: 0.000000
Episode 600	Average Score: 0.000000
Episode 700	Average Score: 0.000000
Episode 800	Average Score: 0.000000
Episode 900	Average Score: 0.013900
Episode 1000	Average Score: 0.000000
Episode 1100	Average Score: 0.000000
Episode 1200	Average Score: 0.000000
Episode 1300	Average Score: 0.000000
Episode 1400	Average Score: 0.009700
Episode 1500	Average Score: 0.029700
Episode 1600	Average Score: 0.019000
Episode 1700	Average Score: 0.053900
Episode 1800	Average Score: 0.041400
Episode 1900	Average Score: 0.020600
Episode 2000	Average Score: 0.062400
Episode 2100	Average Score: 0.064100
Episode 2200	Average Score: 0.075100
Episode 2300	Average Score: 0.084300
Episode 2400	Average Score: 0.063300
Episode 2478	Average Score: 0.102900

In [ ]:
# Plot the scores over learning time. 

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores, label='score')
plt.plot([0, len(scores)], [SCORE_ACCEPTANCE_THRESHOLD, SCORE_ACCEPTANCE_THRESHOLD], 
         linestyle='dashed', 
         label='acceptance threshold')
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.legend()
plt.show()

### 5. Test our arm's performance. 

In [ ]:
# Load the weights from file. 
for idx in range(num_agents): 
    agent.agents[idx].actor_local.load_state_dict(torch.load(f'../model/trained_actor_{idx}.pt'))

env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states)                          # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print(f'Total score this episode: {round(np.mean(scores),3)}')

In [ ]:
env.close()